In [1]:
# alpha vantage API key KFHTNBCI7OJAQVDR

In [34]:
import os
import requests
import pandas as pd
import time
from io import StringIO

In [53]:
def fetch_alpha_vantage_extended_data(symbol, api_key, interval='1min', adjusted='false', extended_hours='true', datatype='csv'):
    all_data = pd.DataFrame()
    
    # Generate a list of months for the past 2 years
    current_year = pd.Timestamp.now().year
    current_month = pd.Timestamp.now().month
    
    for year in range(current_year - 0, current_year + 1):
        start_month = 1 if year < current_year else 1
        end_month = 12 if year < current_year else current_month
        for month in range(start_month, end_month + 1):
            month_str = f"{year:04d}-{month:02d}"
            url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval={interval}&month={month_str}&apikey={api_key}&adjusted={adjusted}&extended_hours={extended_hours}&datatype={datatype}'
            response = requests.get(url)
            if response.status_code != 200:
                print(f"Failed to retrieve data for {symbol} - {month_str}: {response.status_code}")
                continue
            
            # Print the raw response content for debugging
            print(f"Raw response for {symbol} - {month_str}:\n{response.text[:1000]}")  # Print the first 1000 characters of the response
            
            # Parse CSV data from the response
            try:
                df = pd.read_csv(StringIO(response.text))
                if df.empty:
                    print(f"No data found for {symbol} - {month_str}")
                    continue
                
                print(f"Data for {symbol} - {month_str} (first 5 rows):")
                print(df.head())
                
                # Combine data
                all_data = pd.concat([all_data, df], axis=0)
                print(f"Fetched data for {symbol} - {month_str}")
            except pd.errors.ParserError as e:
                print(f"Error parsing data for {symbol} - {month_str}: {e}")
            
            time.sleep(12)  # Respect rate limits
    
    # Process data to ensure correct formatting
    if not all_data.empty:
        # Inspect columns to find the correct column names
        print(f"Combined data columns for {symbol}: {all_data.columns}")
        
        # Adjust the column names based on the actual CSV structure
        if 'time' in all_data.columns:
            all_data['timestamp'] = pd.to_datetime(all_data['time'])
        elif 'timestamp' in all_data.columns:
            all_data['timestamp'] = pd.to_datetime(all_data['timestamp'])
        # Add additional elif cases if needed to handle other possible column names
        
        all_data = all_data.sort_values('timestamp')
        all_data = all_data.rename(columns={
            'time': 'timestamp',
            'open': 'open',
            'high': 'high',
            'low': 'low',
            'close': 'close',
            'volume': 'volume'
        })
        all_data.set_index('timestamp', inplace=True)
    
    return all_data


In [54]:
def fetch_all_stocks_data(stocks, api_key):
    all_data = {}
    for symbol in stocks:
        print(f'Fetching data for {symbol}')
        df = fetch_alpha_vantage_extended_data(symbol, api_key)
        if not df.empty:
            all_data[symbol] = df
            # Save the data to a CSV file
            filename = f'data/{symbol}_data.csv'
            df.to_csv(filename)
            print(f'Data for {symbol} saved to {filename}')
        time.sleep(12)  # Respect rate limits
    return all_data


In [55]:
api_key = 'CH0067CVA7FOCUP6'

stocks = pd.read_csv('finviz_stocks.csv')['Stock Symbol'].tolist()

# Create directory if it does not exist
if not os.path.exists('data'):
    os.makedirs('data')

data = fetch_all_stocks_data(stocks[:1], api_key)

print(f"Data fetched for {len(data)} stocks.")

Fetching data for AAOI
Raw response for AAOI - 2024-01:
{
    "Information": "Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits."
}
Data for AAOI - 2024-01 (first 5 rows):
                                                   {
0      "Information": "Thank you for using Alpha ...
1                                                  }
Fetched data for AAOI - 2024-01
Raw response for AAOI - 2024-02:
{
    "Information": "Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits."
}
Data for AAOI - 2024-02 (first 5 rows):
                                                   {
0      "Information": "Thank you for using Alpha ...
1                                                  }
F

KeyError: 'timestamp'